In [1]:
from ltr.client.elastic_client import ElasticClient
client = ElasticClient()

In [ ]:
from ltr import download
download()


In [5]:
from ltr.index import rebuild_tmdb
rebuild_tmdb(client)

Reindexing...
Deleted index tmdb [Status: 200]
Created index tmdb [Status: 200]
Streaming Bulk index DONE tmdb [Status: 201]
Done


In [2]:
config = {
    "featureset": {
        "features": [
            {
                "name": "title_bm25",
                "params": [],
                "template": {
                    "match": {
                        "title": "{{keywords}}"
                    } 
                }
            },
            {
                "name": "text_all_bm25",
                "params": [],
                "template": {
                    "match": {
                        "text_all": "{{keywords}}"
                    } 
                }
            },
            {
                "name": "release_year",
                "params": [],
                "template": {
                    "function_score": {
                        "field_value_factor": {
                            "field": "release_year",
                            "missing": 2000
                        },
                        "query": { "match_all": {} }
                    }
                }
            }
        ]
    }
}


from ltr import setup
setup(client, config=config, featureset='genome')

Removed Default LTR feature store [Status: 200]
Initialize Default LTR feature store [Status: 200]
Create genome feature set [Status: 201]


In [2]:
from ltr.log import judgments_to_training_set
trainingSet = judgments_to_training_set(client,
                                        judgmentInFile='data/genome_judgments.txt', 
                                        trainingOutFile='data/genome_judgments_train.txt', 
                                        featureSet='genome')

Recognizing 1128 queries...
Parsing QID 100
Parsing QID 200
Parsing QID 300
Parsing QID 400
Parsing QID 500
Parsing QID 600
Parsing QID 700
Parsing QID 800
Parsing QID 900
Parsing QID 1000
Parsing QID 1100
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for 007 (0/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for 007 (series) (1/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for 18th century (2/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA f

Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for alcoholism (41/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for alien (42/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for alien invasion (43/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for aliens (44/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for allegory (45/1128)
Searching tmdb - [{'terms': {

Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for assassination (81/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for assassins (82/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for astronauts (83/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for atheism (84/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for atmospheric (85/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'te

Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for beer (123/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for berlin (124/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for best of 2005 (125/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for best war films (126/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for betrayal (127/1128)
Searching tmdb - [{'ter

Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for british (163/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for british comedy (164/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for broadway (165/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for brothers (166/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for brutal (167/1128)
Searching tmdb - [{'term

...

Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for toys (1035/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for tragedy (1036/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for train (1037/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for trains (1038/1128)
Duplicate Doc in qid:1040 105045
Duplicate Doc in qid:1040 105045
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for transformation (1039/1128)
Searching tm

Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for videogame (1076/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for vienna (1077/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for vietnam (1078/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for vietnam war (1079/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for view askew (1080/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{

Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for workplace (1117/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for world politics (1118/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for world war i (1119/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for world war ii (1120/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUIL

Discarded 1146 Keep 467038
Discarded 1148 Keep 468115
Discarded 1152 Keep 469148
Discarded 1154 Keep 470254
Discarded 1157 Keep 471301
Discarded 1159 Keep 472345
Discarded 1161 Keep 473074
Discarded 1166 Keep 473946
Discarded 1172 Keep 475269
Discarded 1175 Keep 476274
Discarded 1176 Keep 477328
Discarded 1177 Keep 478408
Discarded 1178 Keep 479586
Discarded 1181 Keep 480445
Discarded 1186 Keep 481464
Discarded 1189 Keep 482516
Discarded 1191 Keep 483643
Discarded 1194 Keep 484722
Discarded 1195 Keep 485740
Discarded 1200 Keep 486858
Discarded 1204 Keep 487915
Discarded 1206 Keep 488965
Discarded 1209 Keep 490007
Discarded 1212 Keep 491040
Discarded 1214 Keep 492111
Discarded 1216 Keep 492695
Discarded 1219 Keep 494007
Discarded 1220 Keep 495003
Discarded 1222 Keep 496024
Discarded 1224 Keep 496636
Discarded 1228 Keep 497599
Discarded 1230 Keep 498643
Discarded 1233 Keep 499651
Discarded 1234 Keep 500653
Discarded 1234 Keep 501625
Discarded 1238 Keep 502600
Discarded 1240 Keep 503613
D

Discarded 2162 Keep 873165
Discarded 2167 Keep 874165
Discarded 2171 Keep 875209
Discarded 2172 Keep 876255
Discarded 2179 Keep 877327
Discarded 2183 Keep 878345
Discarded 2187 Keep 879356
Discarded 2189 Keep 880397
Discarded 2190 Keep 881440
Discarded 2191 Keep 882504
Discarded 2194 Keep 883585
Discarded 2196 Keep 884624
Discarded 2200 Keep 885662
Discarded 2202 Keep 886742
Discarded 2206 Keep 887815
Discarded 2211 Keep 888850
Discarded 2213 Keep 889816
Discarded 2218 Keep 890881
Discarded 2221 Keep 891930
Discarded 2223 Keep 892977
Discarded 2227 Keep 893821
Discarded 2230 Keep 894900
Discarded 2233 Keep 896403
Discarded 2234 Keep 897498
Discarded 2238 Keep 898526
Discarded 2238 Keep 899694
Discarded 2240 Keep 900699
Discarded 2244 Keep 901742
Discarded 2246 Keep 902750
Discarded 2250 Keep 903795
Discarded 2252 Keep 904844
Discarded 2254 Keep 905946
Discarded 2256 Keep 906982
Discarded 2259 Keep 908033
Discarded 2263 Keep 909111
Discarded 2265 Keep 910151
Discarded 2269 Keep 911224
D

In [5]:
from ltr.train import train
trainLog = train(client,
                 trainingInFile='data/genome_judgments_train.txt',
                 metric2t='NDCG@10',
                 featureSet='genome',
                 modelName='genome')

print()
print("Impact of each feature on the model")
for ftrId, impact in trainLog.impacts.items():
    print("{} - {}".format(ftrId, impact))
    
print("trainLog Metric %s" % trainLog.metric())

Running java -jar data/RankyMcRankFace.jar -ranker 6 -metric2t NDCG@10 -tree 10 -leaf 10 -train data/genome_judgments_train.txt -save data/genome_model.txt 
DONE
Delete model genome: 200
Created Model genome [Status: 201]

Impact of each feature on the model
2 - 4795425.7473736005
1 - 1882557.8524923166
3 - 811584.225853294
trainLog Metric 0.5604
